In [11]:
import pandas as pd
from datasets import load_dataset

data = load_dataset("csv", data_files="megablunders.csv", )

In [12]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased")

In [ ]:
# Define label list based on your grammar error categories
label_list = ["AGREE", "DM", "MM", "ROS", "PR", "PAR", "FRAG", "CASE", "NONE"]
label_to_id = {label: i for i, label in enumerate(label_list)}

# Function to convert string labels to numerical indices
def convert_labels(examples):
    # Make sure the column name matches your dataset's label column
    # If your label column is not named "label", change it accordingly
    examples["label"] = [label_to_id[label] for label in examples["label"]]
    return examples

# Apply the conversion to your dataset
data = data.map(convert_labels, batched=True)

# Verify the conversion worked
print(f"Converted labels: {data['train']['label'][:5]}")

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

Original labels: [1, 6, 4, 0, 5]


KeyError: "Column labels not in the dataset. Current columns in the dataset: ['text', 'label']"

In [ ]:
data['train'][:5]

{'text': ['By dropping a game to the pathetic Tampa Bay Devil Rays, we may be on the verge of another Red Sox collapse.',
  'Although, if history is any indication, the tease could still go on for several weeks.',
  'The Sox almost never go down uneventfully, which is the most frustrating thing for their legion of fans.',
  'Because of the accumulated bad karma that hangs over the franchise, the skill of the players in any given year seem to be irrelevant to the outcome.',
  'The team not only has squandered huge leads but also it has come roaring from behind only to fall short.to fall short.'],
 'label': [1, 6, 4, 0, 5],
 'labels': [1, 6, 4, 0, 5]}

In [ ]:
def tokenize(examples):
   return tokenizer(examples["text"], padding="max_length", truncation=True)

data = data.map(tokenize, batched=True)

Map:   0%|          | 0/185 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased", num_labels=9)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import numpy as np
import evaluate 

metric = evaluate.load("accuracy")

def compute_metrics(eval_prod):
    loigts, labels = eval_prod
    
    predictions = np.argmax(loigts, axis=-1)
    return metric.compute(predictions=predictions, labels=labels)

In [ ]:
from transformers import TrainingArguments

trainingArguments = TrainingArguments(
    output_dir="megablunders",
    eval_strategy="epoch", 
    push_to_hub=False
)

In [ ]:
from transformers import Trainer


from transformers import Trainer
from datasets import DatasetDict

# Split the training data into train and test sets
train_test_split = data["train"].train_test_split(test_size=0.2, seed=42)
data = DatasetDict({
    "train": train_test_split["train"],
    "test": train_test_split["test"]
})

trainer = Trainer(
    model=model,
    args=trainingArguments,
    train_dataset=data["train"],
    eval_dataset=data["test"],
    compute_metrics=compute_metrics
)

trainer.train()

ValueError: too many dimensions 'str'